<a href="https://colab.research.google.com/github/HKU-BAL/Clair3/blob/main/colab/clair3_pacbio_hifi_quick_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Clair3 pacbio hifi demo</h1>

The following is the document describing the calling workflow of `Clair3`. Clair3 is an open-source project available at https://github.com/HKU-BAL/Clair3.

We will demonstrate Clair3 long-read variant calling workflow, including pileup calling and full-alignment calling.

The document you are reading is not a static web page, but an interactive environment called a Colab notebook that lets you write and execute code. You can inspect, modify, and run any of the code on this page. 

***Please note that current workflow is using google colab resource, we only suggest using small datasets for benchmarking.***

## Installation

Before getting started with how `Clair3` works, we will install it into the Colab environment. This will enable us to both inspect and run the code. To install the `Clair3`, run the code cell below by clicking the "play" icon to the left or pressing on the cell or type `<ctrl>-<enter>` to run the cell.

### Build an anaconda virtual environment

In [1]:
# build an anaconda virtual environment
!mkdir -p /content
%cd /content
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x ./Miniconda3-latest-Linux-x86_64.sh 
!bash ./Miniconda3-latest-Linux-x86_64.sh -f -b -p /usr/local

/content
--2021-06-16 11:35:30--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94235922 (90M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  89.87M   184MB/s    in 0.5s    

2021-06-16 11:35:30 (184 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [94235922/94235922]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - brotlipy==0.7.0=py38h27cfd23_1003
    - ca-certificates==2020.10.14=0
    - certifi==2020.6.20=pyhd3eb1b0_3
    - cffi==1.14.3=py38h261ae71_2
    - chardet==3.0.4=py38h06a4308_1003
    - conda-package-

### Use conda environment to install python packages

In [2]:
# create and activate an environment named clair3
%%bash
conda create -n clair3 python=3.6.10 -y
source activate clair3

conda install -c conda-forge pypy3.6 -y
pypy3 -m ensurepip
pypy3 -m pip install intervaltree==3.0.2
pypy3 -m pip install mpmath==1.2.1

# install python packages in environment
pip3 install tensorflow==2.2.0
pip3 install intervaltree==3.0.2  tensorflow-addons==0.11.2 tables==3.6.1
conda install -c anaconda pigz==2.4 -y
conda install -c conda-forge parallel=20191122 zstd=1.4.4 -y
conda install -c conda-forge -c bioconda samtools=1.10 -y
conda install -c conda-forge -c bioconda whatshap=1.0 -y
cd /content
git clone https://github.com/HKU-BAL/Clair3.git
cd /content/Clair3
# download pre-trained models
mkdir models
wget --quiet http://www.bio8.cs.hku.hk/clair3/clair3_models/clair3_models.tar.gz 
tar -zxvf clair3_models.tar.gz -C ./models

Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/clair3

  added / updated specs:
    - python=3.6.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |            1_gnu          22 KB
    ca-certificates-2021.5.25  |       h06a4308_1         112 KB
    certifi-2021.5.30          |   py36h06a4308_0         139 KB
    ld_impl_linux-64-2.35.1    |       h7274673_9         586 KB
    libgcc-ng-9.3.0            |      h5101ec6_17         4.8 MB
    libgomp-9.3.0              |      h5101ec6_17         311 KB
    libstdcxx-ng-9.3.0         |      hd4cf53a_17         3.1 MB
    openssl-1.1.1k             |       h27cfd23_0         2.5 MB
    pip-21.1.2                 |   py36h06a4308_0         



==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda




==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda




==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda




==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda




==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda




==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest vers

## PacBio HiFi quick demo

Here is a quick demo for the PacBio HiFi variant calling using GIAB HG003 chromosome 20 data. We select a small chunked regions (chr20:100000-300000) to run Clair3 workflow.

In [3]:
%%bash
source activate clair3
PLATFORM="hifi"
INPUT_DIR="/content/data"
OUTPUT_DIR="/content/clair3_hifi_demo"
THREADS=4        
mkdir -p ${INPUT_DIR}
mkdir -p ${OUTPUT_DIR}

# download quick demo data
# GRCh38_no_alt Reference
wget --quiet -P ${INPUT_DIR} http://www.bio8.cs.hku.hk/clair3/demo/quick_demo/pacbio_hifi/GRCh38_no_alt_chr20.fa
wget --quiet -P ${INPUT_DIR} http://www.bio8.cs.hku.hk/clair3/demo/quick_demo/pacbio_hifi/GRCh38_no_alt_chr20.fa.fai
# BAM chr20:100000-300000
wget --quiet -P ${INPUT_DIR} http://www.bio8.cs.hku.hk/clair3/demo/quick_demo/pacbio_hifi/HG003_chr20_demo.bam
wget --quiet -P ${INPUT_DIR} http://www.bio8.cs.hku.hk/clair3/demo/quick_demo/pacbio_hifi/HG003_chr20_demo.bam.bai
# GIAB Truth VCF and BED
wget --quiet -P ${INPUT_DIR} http://www.bio8.cs.hku.hk/clair3/demo/quick_demo/pacbio_hifi/HG003_GRCh38_chr20_v4.2.1_benchmark.vcf.gz
wget --quiet -P ${INPUT_DIR} http://www.bio8.cs.hku.hk/clair3/demo/quick_demo/pacbio_hifi/HG003_GRCh38_chr20_v4.2.1_benchmark.vcf.gz.tbi
wget --quiet -P ${INPUT_DIR} http://www.bio8.cs.hku.hk/clair3/demo/quick_demo/pacbio_hifi/HG003_GRCh38_chr20_v4.2.1_benchmark_noinconsistent.bed

REF="GRCh38_no_alt_chr20.fa"
BAM="HG003_chr20_demo.bam"

CONTIGS="chr20"
START_POS=100000
END_POS=300000
echo -e "${CONTIGS}\t${START_POS}\t${END_POS}" > ${INPUT_DIR}/quick_demo.bed

# run Clair3 using one command
cd /content/Clair3
./run_clair3.sh \
  --bam_fn=${INPUT_DIR}/${BAM} \
  --ref_fn=${INPUT_DIR}/${REF} \
  --threads=${THREADS} \
  --platform=${PLATFORM} \
  --model_path=`pwd`"/models/${PLATFORM}" \
  --output=${OUTPUT_DIR} \
  --bed_fn=${INPUT_DIR}/quick_demo.bed


[INFO] BAM FILE PATH: /content/data/HG003_chr20_demo.bam
[INFO] REFERENCE FILE PATH: /content/data/GRCh38_no_alt_chr20.fa
[INFO] MODEL PATH: /content/Clair3/models/hifi
[INFO] OUTPUT FOLDER: /content/clair3_hifi_demo
[INFO] PLATFORM: hifi
[INFO] THREADS: 4
[INFO] BED FILE PATH: /content/data/quick_demo.bed
[INFO] VCF FILE PATH: EMPTY
[INFO] CONTIGS: EMPTY
[INFO] SAMTOOLS PATH: samtools
[INFO] PYTHON PATH: python3
[INFO] PYPY PATH: pypy3
[INFO] PARALLEL PATH: parallel
[INFO] WHATSHAP PATH: whatshap
[INFO] CHUNK SIZE: 5000000
[INFO] CHUNK NUM: 0
[INFO] FULL ALIGN PROPORTION: 0.3
[INFO] FULL ALIGN RERFERENCE PROPORTION: 0.3
[INFO] USER DEFINED SNP THRESHOLD: 0.0
[INFO] USER DEFINED INDEL THRESHOLD: 0.0
[INFO] ENABLE FILEUP ONLY CALLING: False
[INFO] ENABLE FAST MODE CALLING: False
[INFO] ENABLE PRINTING REFERENCE CALLS: False
[INFO] ENABLE OUTPUT GVCF: False
[INFO] ENABLE HAPLOID PRECISE MODE: False
[INFO] ENABLE HAPLOID SENSITIVE MODE: False
[INFO] ENABLE INCLUDE ALL CTGS CALLING: False


## Benchmarking

We used `hap.py` version 0.3.12 to assess the variant calls against GIAB truth set. The `hap.py`
program can be installed using conda. 

In [4]:
# install hap.py without root privileges using conda
%%bash
conda config --add channels defaults
conda config --add channels bioconda
conda config --add channels conda-forge
conda create -n happy-env -c bioconda hap.py -y
conda install -c bioconda rtg-tools -y



Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/happy-env

  added / updated specs:
    - hap.py


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            1_gnu          22 KB  conda-forge
    bcftools-1.10.2            |       h4f4756c_3         773 KB  bioconda
    bedtools-2.30.0            |       h7d7f7ad_1        17.9 MB  bioconda
    bx-python-0.8.9            |   py27h54ae540_2        1021 KB  bioconda
    certifi-2019.11.28         |   py27h8c360ce_1         149 KB  conda-forge
    gsl-2.6                    |       he838d99_2         3.2 MB  conda-forge
    hap.py-0.3.14              |   py27h5c5a3



==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda




==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda




In [6]:
%%bash
# run hap.py for benchmarking
INPUT_DIR="/content/data"
OUTPUT_DIR="/content/clair3_hifi_demo"
BASELINE_VCF_FILE_PATH="HG003_GRCh38_chr20_v4.2.1_benchmark.vcf.gz"
BASELINE_BED_FILE_PATH="HG003_GRCh38_chr20_v4.2.1_benchmark_noinconsistent.bed"
OUTPUT_VCF_FILE_PATH="merge_output.vcf.gz"
REF="GRCh38_no_alt_chr20.fa"
CONTIGS="chr20"
START_POS=100000
END_POS=300000
THREADS=4
source activate happy-env
conda install -c bioconda rtg-tools -y 
hap.py \
    ${INPUT_DIR}/${BASELINE_VCF_FILE_PATH} \
    ${OUTPUT_DIR}/${OUTPUT_VCF_FILE_PATH} \
    -f "${INPUT_DIR}/${BASELINE_BED_FILE_PATH}" \
    -r "${INPUT_DIR}/${REF}" \
    -o "${OUTPUT_DIR}/happy" \
    -l ${CONTIGS}:${START_POS}-${END_POS} \
    --engine=vcfeval \
    --threads="${THREADS}" \
    --pass-only

Solving environment: ...working... done

# All requested packages already installed.

Hap.py 
Benchmarking Summary:
  Type Filter  TRUTH.TOTAL  TRUTH.TP  TRUTH.FN  QUERY.TOTAL  QUERY.FP  QUERY.UNK  FP.gt  FP.al  METRIC.Recall  METRIC.Precision  METRIC.Frac_NA  METRIC.F1_Score  TRUTH.TOTAL.TiTv_ratio  QUERY.TOTAL.TiTv_ratio  TRUTH.TOTAL.het_hom_ratio  QUERY.TOTAL.het_hom_ratio
 INDEL    ALL           59        59         0          102         0         43      0      0            1.0               1.0        0.421569              1.0                     NaN                     NaN                   4.090909                   3.950000
 INDEL   PASS           59        59         0          102         0         43      0      0            1.0               1.0        0.421569              1.0                     NaN                     NaN                   4.090909                   3.950000
   SNP    ALL          402       402         0          467         0         64      0      0 

2021-06-16 11:46:03,339 WARNING  No reference file found at default locations. You can set the environment variable 'HGREF' or 'HG19' to point to a suitable Fasta file.
[I] Total VCF records:         84252
[I] Non-reference VCF records: 84252
[I] Total VCF records:         567
[I] Non-reference VCF records: 567
2021-06-16 11:46:05,081 WARNING  starting at chr20:99999
2021-06-16 11:46:05,253 WARNING  No calls for location chr20:100000-300000 in query!
2021-06-16 11:46:05,253 WARNING  Creating template for vcfeval. You can speed this up by supplying a SDF template that corresponds to /content/data/GRCh38_no_alt_chr20.fa
